In [1]:
using JSON, SDDP,DataFrames
using Plots, StatPlots
const mm = Plots.mm
const pt = Plots.pt
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm)
gr()

Plots.GRBackend()

In [39]:
# Plot main figure
function getresults(results, key, scalefactor=1.0)
    Q = [0.01, 0.1, 0.5, 0.9, 0.99, 0.01, 0.99]
    g2 = hcat([scalefactor*r[key] for r in results]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    DataFrame(week=1:size(g3, 1), x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5],x05=g3[:,6],x95=g3[:,7])
end
function plotpowder(data, key, ylims, ylabel,title,scalefactor=1.0;plotline::Int=0)
    blue = getresults(data, key, scalefactor)
    plot()
    plot!(blue, :week, :x10, fill=(:x90, "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
    plot!(blue, :week, :x25, fill=(:x75, "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)   
    plot!(blue, :week, :x50, c="#00467F", w=2)
    if plotline > 0
        plot!(scalefactor*data[plotline][key], c="red", w=3)
    end
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xlims=(1,52), xticks=(xticks, xticklabels), xlabel="")
    plot!(size=(500,300))
end

function plotall(data; plotprice=false,plotline=0)
    for d in data
        d[:contracted_proportion] = fill(0.0, 52)
        d[:milk_production] = fill(0.0, 52)
        d[:milk_sold] = fill(0.0, 52)
        for t in 1:52
            d[:milk_production][t] = sum(d[:milk][1:t]) 
            d[:milk_sold][t] = sum(d[:milk_sales][1:t]) 
            d[:contracted_proportion][t] = d[:milk_sold][t] ./ sum(d[:milk])
        end
    end
    if !plotprice
        plot(
            plotpowder(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)",plotline=plotline),
            plotpowder(data, :P, (0,3500), "Pasture Cover\n(kg/Ha)","(b)",plotline=plotline),
            plotpowder(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7,plotline=plotline),
            plotpowder(data, :milk_production, (0,2000), "Milk Production\n(kg)", "(d)",plotline=plotline),
            plotpowder(data, :milk_sold, (0,2000), "Milk Sales\n(kg)", "(e)",plotline=plotline),
            plotpowder(data, :contracted_proportion, (0, 1.25), "Proportion of Milk Contracted\n(fraction)", "(f)", plotline=plotline),
            layout=(2,3), size=(1500,600)
        )
    else
        parameters = JSON.parsefile("model.parameters.json")
        function futures_contribution(gt, w, auction)
            y = 0.0
            for i in (auction+1):length(w)
                y += w[i] * (0.96 ^ (i - auction) * gt + 0.252 * sum(0.96^(j-1) for j in 1:(i-auction)))
            end
            y
        end
        for n in data
            n[:gdt] = [p[1] for p in n[:price]]
            n[:accumulated] = [p[2] for p in n[:price]]
            n[:futures] = zeros(Float64, 52)
            for stage in 1:52
                auction = findlast(x->x<=stage, parameters["auction_weeks"])
                n[:futures][stage] = n[:accumulated][stage] + futures_contribution(n[:gdt][stage], parameters["sales_curve"], auction)
            end
        end
        plot(
            plotpowder(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)",plotline=plotline),
            plotpowder(data, :P, (0,3500), "Pasture Cover\n(kg/Ha)","(b)",plotline=plotline),
            plotpowder(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7,plotline=plotline),
            plotpowder(data, :milk_production, (0,2000), "Milk Production\n(kg)", "(d)",plotline=plotline),
            plotpowder(data, :milk_sold, (0,2000), "Milk Sales\n(kg)", "(e)",plotline=plotline),
            plotpowder(data, :contracted_proportion, (0, 1.25), "Proportion of Milk Contracted\n(fraction)", "(f)", plotline=plotline),
            plotpowder(data, :gdt, (0, 10), "Spot Price\n(\$/kg)", "(h)",plotline=plotline),
            plotpowder(data, :accumulated, (0, 10), "Accumulated Price\n(\$/kg)", "(h)",plotline=plotline),
            plotpowder(data, :futures, (0, 10), "Futures Price\n(\$/kg)", "(i)",plotline=plotline),
            layout=(3,3), size=(1500,900)
        )
    end
end

function plotdensity!(data, label, color, w=1, a=1.0, densityflag=true)
    x = [sim[:objective] - 3536 for sim in data]
    if densityflag
        density!(x, label=label, c=color, w=w, alpha=a)
    else
        vline!([mean(x)], label="", c=color, w=w, alpha=a, linestyle=:dash)
    end
end

plotdensity! (generic function with 4 methods)

In [81]:
neutral = SDDP.load("NeutralWithContracting.results")[1:2000];
params = JSON.parsefile("neutral_with_contracting.json")

for n in neutral
    n[:rainfall] = [params["niwa_data"][t][n[:noise][t]]["rainfall"] for t in 1:52]
    n[:evapotranspiration] = [params["niwa_data"][t][n[:noise][t]]["evapotranspiration"] for t in 1:52]
end
plot(
plotpowder(neutral, :price, (2, 10), "Milk Price Price\n(\$/kg)", "(a)"),
plotpowder(neutral, :evapotranspiration, (0, 50), "Evapotranspiration\n(mm/Week)", "(b)"),
plotpowder(neutral, :rainfall, (0, 300), "Rainfall\n(mm/Week)", "(c)"),
    layout=(1,3), size=(1500,300)
)
savefig("price.pdf")

println("Neutral")
plotall(neutral)
savefig("neutral_no_selling.pdf")

println("Averse")
averse = SDDP.load("AverseNoContracting.results")[1:2000];
plotall(averse)
savefig("averse_no_selling.pdf")

println("Selling")
averse_sell = SDDP.load("AverseWithContracting.results")[1:2000];
plotall(averse_sell)
savefig("averse_selling.pdf")

println("High")
high_averse_sell = SDDP.load("HighAverseWithContracting.results")[1:2000];
plotall(high_averse_sell)
savefig("high_averse_selling.pdf")

Neutral
Averse
Selling
High


LoadError: [91mBoundsError[39m

In [77]:
low_averse_sell = SDDP.load("LowAverseWithContracting.results")[1:2000];
plotall(low_averse_sell)
savefig("low_averse_selling.pdf")

In [140]:
neutral = SDDP.load("NeutralWithContracting.results");
averse = SDDP.load("AverseNoContracting.results");
low_averse_sell = SDDP.load("LowAverseWithContracting.results");
averse_sell = SDDP.load("AverseWithContracting.results");
high_averse_sell = SDDP.load("HighAverseWithContracting.results");

In [150]:
N = 2000
scale = 1.5
plot(
    title="Distribution in Operating Profit.\nAV@R fraction = 25\%",
    xlabel="Operating Profit (\$/Ha)\n",
    ylabel="Smoothed Density",
    ytick=false,
    top_margin=10mm, right_margin=5mm, bottom_margin=10mm, 
    size=(scale*500,scale*300),
    legend=:topright, 
#     xlims=(-4000, 12000),
    grid=false
)

for (data, key, colour, alpha) in [
#         (neutral, "Risk-Neutral", "#00467F", 0.5),
#         (averse[1:N], "Medium-Risk", "#00467F", 1.0),
        (low_averse_sell, "Low Risk Averse", "red", 0.4),
        (averse_sell, "Medium Risk Averse", "red", 0.7),
        (high_averse_sell, "High Risk Averse", "red", 1.0),
        
        (low_averse_sell[1:N], "Low Risk Averse", "green", 0.4),
        (averse_sell[1:N], "Medium Risk Averse", "green", 0.7),
        (high_averse_sell[1:N], "High Risk Averse", "green", 1.0)
    ]
    plotdensity!(data, key, colour,3, alpha, true)
    plotdensity!(data, "", colour,3, alpha, false)
    plot!(legend=false)
end

savefig("density_comparison.pdf")

In [35]:
averse_2 = SDDP.load("AverseWithContracting2.results")[1:1000];
plotall(averse_2)
savefig("averse_2.pdf")

In [72]:

# neutral = SDDP.load("NeutralWithContracting.results")[1:2000];
# quantile([n[:price][end] for n in neutral], [0.0, 0.25, 0.5, 0.75, 1.0])
quantile([n[:accumulated][end] for n in averse_2], [0.0, 0.25, 0.5, 0.75, 1.0])

5-element Array{Float64,1}:
 3.50619
 5.52861
 6.12845
 6.70705
 9.23457

In [36]:
averse_2 = SDDP.load("AverseWithContracting2.results")[1:1000];
parameters = JSON.parsefile("model.parameters.json")
function futures_contribution(gt, w, auction)
    y = 0.0
    for i in (auction+1):length(w)
        y += w[i] * (0.96 ^ (i - auction) * gt + 0.252 * sum(0.96^(j-1) for j in 1:(i-auction)))
    end
    y
end
for n in averse_2
    n[:gdt] = [p[1] for p in n[:price]]
    n[:accumulated] = [p[2] for p in n[:price]]
    n[:futures] = zeros(Float64, 52)
    for stage in 1:52
        auction = findlast(x->x<=stage, parameters["auction_weeks"])
        n[:futures][stage] = n[:accumulated][stage] + futures_contribution(n[:gdt][stage], parameters["sales_curve"], auction)
    end
end
plot(
    plotpowder(averse_2, :gdt, (0, 10), "Spot Price\n(\$/kg)", "(a)",plotline=828),
    plotpowder(averse_2, :accumulated, (0, 10), "Accumulated Price\n(\$/kg)", "(b)",plotline=828),
    plotpowder(averse_2, :futures, (0, 10), "Futures Price\n(\$/kg)", "(c)",plotline=828),
    layout=(1,3), size=(1500,300)
)
savefig("price_2.pdf")

In [44]:
plot()
plotdensity!(averse_2, "Risk-Averse", "#00467F",3, 0.5, true)
plotdensity!(averse_2, "Risk-Averse", "#00467F",3, 0.5, false)

plot!(ytick=false, xlabel="Operating Profit (\$/Ha)\n", ylabel="Smoothed Density")
plot!(size=(6 * 150,3 * 150), right_margin=5Plots.mm, bottom_margin=10Plots.mm, legend=:topright, xlims=(-4000, 12000),grid=false)
savefig("density_comparison2.pdf")

In [294]:
open("price_simulations.json", "w") do io
    write(io, JSON.json([n[:futures] for n in averse_2]))
end

937771

In [131]:
function cvar(x::Vector{Float64}, b::Float64)
    mean(x[x .<= quantile(x, b)])
end
function cvar(x::Vector{Dict{Symbol, Any}},b)
    cvar(Float64[s[:objective] for s in x], b)
end
function avar(x, λ, β)
   λ * cvar(x, 1.0) + (1 - λ) * cvar(x, β) 
end
@show avar(high_averse_sell, 0.25, 0.25)
@show avar(averse_sell, 0.25, 0.25)
@show avar(neutral, 0.25, 0.25)

@show avar(high_averse_sell, 0.5, 0.25)
@show avar(averse_sell, 0.5, 0.25)
@show avar(neutral, 0.5, 0.25)

avar(high_averse_sell, 0.25, 0.25) = 5054.012890974575
avar(averse_sell, 0.25, 0.25) = 5590.091798142579
avar(neutral, 0.25, 0.25) = 4776.544643164486
avar(high_averse_sell, 0.5, 0.25) = 5214.534148254148
avar(averse_sell, 0.5, 0.25) = 5734.717452305817
avar(neutral, 0.5, 0.25) = 5231.0085967781015


5231.0085967781015

In [137]:
function meanobj(data)
    round(Int, median(Float64[s[:objective] for s in data])) - 3536
#     round(Int, cvar(Float64[s[:objective] for s in data] - 3536, 0.0))
end
meanobj(neutral), meanobj(averse), meanobj(low_averse_sell), meanobj(averse_sell), meanobj(high_averse_sell)

(2640, 2491, 2638, 2518, 2015)

In [59]:
function getmean(data)
    mean(sim[:objective] - 3536 for sim in data), std(sim[:objective] - 3536 for sim in data)
end
@show getmean(neutral)
@show getmean(averse)
@show getmean(averse_sell)
@show getmean(high_averse_sell)

getmean(neutral) = (2603.9365040053235, 1412.2885658140735)
getmean(averse) = (2457.7203779547917, 1342.3236657111693)
getmean(averse_sell) = (2487.9687606322977, 435.988355234436)
getmean(high_averse_sell) = (1999.5766628132917, 490.2626396824679)


(1999.5766628132917, 490.2626396824679)

In [62]:
function undoformatting(x)
    if x[end] == 'K'
        return 1_000.0 * parse(Float64, String(x[1:(end-1)]))
    elseif x[end] == 'M'
        return 1_000_000.0 * parse(Float64, String(x[1:(end-1)]))
    else
        return parse(Float64, x)
    end 
end
# iteration, time, simulation, bound
function getlogdata(filename)
    y = open(filename, "r") do io
        y = Array{Float64}(0, 4)
        for i in 1:14
            readline(io)
        end
        while !eof(io)
            line = readline(io)
            if contains(line, "--")
                break
            end
            items = split(strip(line))
            if length(items) == 1
                break
            end
            z = undoformatting.(items[[4,8,1,2]])'
            y = vcat(y, z)
        end
        y
    end

    y[:, 1] = 1:size(y, 1)
    y
end

r0 = getlogdata("AverseWithContracting.log");
r1 = getlogdata("NeutralWithContracting.log");
# r3 = getlogdata("AverseContractingRib3.log")
# r5 = getlogdata("AverseContractingRib5.log")
# r9 = getlogdata("AverseContractingRib9.log");
r1[1:4,:]

4×4 Array{Float64,2}:
 1.0  19.0  -254034.0  112163.0
 2.0  19.5  -258079.0  112163.0
 3.0  19.5  -256570.0  112163.0
 4.0  19.5  -256542.0  112163.0

In [76]:
# plot(r0[:,1], r0[:, 2])
# plot(r1[:,1], r1[:, 4])
r1[790:800, :]

11×4 Array{Float64,2}:
 790.0  214.2  3708.0  6363.0
 791.0  216.3  5631.0  6363.0
 792.0  217.3  2653.0  6344.0
 793.0  217.6  3210.0  6319.0
 794.0  217.7  5625.0  6319.0
 795.0  218.0  5383.0  6319.0
 796.0  218.4  5565.0  6319.0
 797.0  218.5  4862.0  6319.0
 798.0  218.9  7105.0  6319.0
 799.0  219.4  5284.0  6311.0
 800.0  219.5  4167.0  6311.0

In [70]:
N = 25
plot()
plot!(r3[1:N:size(r3, 1), 2], r3[1:N:size(r3, 1), 4], label="3 Ribs", c="#00467F", alpha=0.4, w=2)
plot!(r5[1:N:size(r5, 1), 2], r5[1:N:size(r5, 1), 4], label="5 Ribs", c="#00467F", alpha=0.7, w=2)
plot!(r9[1:N:size(r9, 1), 2], r9[1:N:size(r9, 1), 4], label="9 Ribs", c="#00467F", alpha=1.0, w=2)
plot!(r0[1:N:size(r0, 1), 2], r0[1:N:size(r0, 1), 4], label="Dynamic", c="#e65100", linestyle=:dot, w=2)
plot!(xlabel="Solution Time (s)\n", ylabel="Risk-adjusted Operating Profit\nUpper Bound (\$/Ha)")
plot!(xlims=(0, 21_600), ylims=(4000, 5000), size=(6*150, 3*150))
savefig("performance_comparison.pdf")